In [9]:
from pathlib import Path
from typing import List

from IPython.display import display
from ipywidgets import widgets

import logging

logger = logging.getLogger()

out = widgets.Output()

from dashboard_funcs import (
    create_start_standalone_command,
    write_shell_file,
    # extract_archive,
    s3_download_file,
    # s3_list_contents,
    write_fews_desktop_shortcut,
    s3_download_directory
)

BUCKET_NAME = "ciroh-rti-hefs-data"

FEWS_BASE_DIR = Path("/opt", "data")
FEWS_INSTALL_DIR = Path("/opt", "fews")

FEWS_BASE_DIR.mkdir(exist_ok=True)

# S3_ROOT_DIR = "standalone_configurations"

In [ ]:
# # List the unique RFCs in the s3 bucket.
# s3_filelist = s3_list_contents(prefix=S3_ROOT_DIR)
# unique_rfcs = list(set([Path(filepath).parts[1] for filepath in s3_filelist]))
unique_rfcs = [
    "ABRFC",
    "APRFC",
    "CBRFC",
    "CNRFC",
    "LMRFC",
    "MARFC",
    "MBRFC",
    "NCRFC",
    "NERFC",
    "NWRFC",
    "OHRFC",
    "SERFC",
    "WGRFC"
]

In [10]:
rfc_dropdown = widgets.Dropdown(
    options=unique_rfcs,
    description='RFC:',
    disabled=False,
    value="MARFC",
)

In [15]:
@out.capture(clear_output=True, wait=True)
def install_fews_standalone(b) -> None:
    """Download standalone configuration from S3 to the working directory."""
    # 1. Download sa from S3
    print("Downloading RFC configuration...This will take a few minutes...")
    s3_download_directory(
        prefix=f"{rfc_dropdown.value}/Config",
        local=FEWS_BASE_DIR
    )
    # 2. Create the bash command to run the standalone configuration
    sa_dir_path = Path(FEWS_BASE_DIR, rfc_dropdown.value)
    bash_command_str = create_start_standalone_command(
        fews_root_dir=FEWS_INSTALL_DIR.as_posix(),
        configuration_dir=sa_dir_path.as_posix()
    )
    # 3. Write the command to start FEWS to a shell script
    shell_script_filepath = Path(sa_dir_path, "start_fews_standalone.sh")
    write_shell_file(shell_script_filepath, bash_command_str)
    # 4. Copy in patch file for the downloaded standalone config.
    print("Downloading patch file...")
    s3_download_file(
        remote_filepath="fews-install/fews-NA-202102-125264-patch.jar",
        local_filepath=Path(sa_dir_path, "fews-NA-202102-125264-patch.jar")
    )
    # 5. Create FEWS desktop shortcut that calls the shell script
    desktop_shortcut_filepath = Path(Path.home(), "Desktop", f"{sa_dir_path.name}.desktop")
    print(f"Creating FEWS desktop shortcut...{desktop_shortcut_filepath}")
    write_fews_desktop_shortcut(desktop_shortcut_filepath, shell_script_filepath)
    print("Installation complete.")
    return

In [ ]:
@out.capture(clear_output=True, wait=True)
def download_historical_data(b) -> None:
    """Download historical data for selected RFC."""
    print("Downloading historical data...")
    s3_download_directory(
        prefix=f"{rfc_dropdown.value}/historicalData",
        local=FEWS_BASE_DIR
    )
    local_path = Path(FEWS_BASE_DIR, rfc_dropdown.value, "historicalData")
    local_path.rename(Path(FEWS_BASE_DIR, rfc_dropdown.value, "cardfiles"))
    pass

In [ ]:
data_button = widgets.Button(description='Download Data')
data_button.on_click(download_historical_data)

In [16]:
config_button = widgets.Button(description='Download Config')
config_button.on_click(install_fews_standalone)

![CIROH Logo](CIROHLogo_200x200.png)

## Welcome to the HEFS-FEWS TEEHR-Hub Dashboard!

This experimental dashboards allows you to download and install pre-existing FEWS stand-alone configurations from the ciroh-rti-hefs-data S3 bucket, and a desktop shortcut to open FEWS.

Configure FEWS for your region of interest:

In [ ]:
display(rfc_dropdown)

In [ ]:
# display(sa_dropdown)

In [ ]:
display(config_button)

In [ ]:
display(data_button)

In [ ]:
display(out)